In [ ]:
import joblib
import numpy as np
import pandas as pd
# Import packages
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Sklearn Packages
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier

# Sklearn Evaluation Metrics
from sklearn import metrics
from sklearn.metrics import mean_squared_error, precision_score, confusion_matrix, accuracy_score

# Visualizes all the columns
pd.set_option('display.max_columns', None)

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
array = joblib.load('Dataset.joblib')


In [ ]:
len(array)

In [ ]:
df = pd.DataFrame(array)

In [ ]:
df['EC'] = df[1].apply(lambda x: 1 if x == 'CONFIRMED' else 1 if x == 'CANDIDATE' else 0 )

In [ ]:
features = df.drop(columns=[0,1,'EC'])
target = df.EC

In [ ]:
medians=features.median(axis=1)

In [ ]:
df_dict = features.to_dict(orient='index')
    

In [ ]:
for x in range(0,len(df_dict)):
    for y in range(2,53256):
        df_dict[x][y] = df_dict[x][y] /medians[x] -1
    

In [ ]:
sum =0
for x in range(0,len(df_dict)):
    for y in range(2,53256):
        if str(df_dict[x][y]) == "nan":
            df_dict[x][y] = -999
            sum = sum + 1
        else:
            df_dict[x][y] = float(df_dict[x][y])

print(sum)

In [ ]:
features = pd.DataFrame.from_dict(df_dict,orient='index')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(features, target, random_state=1, test_size=.25)

In [ ]:
# Evaluation function

def evaluation(y_true, y_pred):
    
# Print Accuracy, Recall, F1 Score, and Precision metrics.
    print('Evaluation Metrics:')
    print('Accuracy: ' + str(metrics.accuracy_score(y_test, y_pred)))
    print('Recall: ' + str(metrics.recall_score(y_test, y_pred)))
    print('F1 Score: ' + str(metrics.f1_score(y_test, y_pred)))
    print('Precision: ' + str(metrics.precision_score(y_test, y_pred)))
    
# Print Confusion Matrix
    print('\nConfusion Matrix:')
    print(' TN,  FP, FN, TP')
    print(confusion_matrix(y_true, y_pred).ravel())
    
# Function Prints best parameters for GridSearchCV
def print_results(results):
    print('Best Parameters: {}\n'.format(results.best_params_))

In [ ]:
tree = DecisionTreeClassifier()

# Fitting Model to the train set
tree.fit(X_train, y_train)

# Predicting on the test set
y_pred = tree.predict(X_test)

# Evaluating model
evaluation(y_test, y_pred)

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from tqdm.notebook import tqdm
from time import sleep
parameter_n_estimators = [500]
for i in tqdm(parameter_n_estimators):
    # Instantiate model
    forest = RandomForestClassifier(n_estimators=i, criterion='gini')
    # Fitting Model to the train set
    forest.fit(X_train, y_train)
    # Predicting on the test set
    y_pred = forest.predict(X_test)

    # Evaluating model
    evaluation(y_test, y_pred)
    print('Tree: %s ' % (i))

In [ ]:
import lightkurve as lk

T_name = 'TIC 145241359'
search_result = lk.search_lightcurve(T_name)
data = []
for x in range(0,17):
    try:
        lc = search_result[x].download() 
        y = lc.flux
        for i in range(1,len(y),10):
            try:
                data.append(float(y[i].value))
            except:
                pass
    except :
        pass

arr2 = pd.DataFrame(data)
medians= arr2.median(axis=0)
print(medians)
arr3 =[]
for x in range(0,53254):
    try:
        arr3.append((arr2[0][x] / medians)-1)
    except:
        arr3.append(-999)

for x in range(0,53254):
    temp = str(arr3[x])
    if temp == 'nan' :
        arr3[x] = -999

In [ ]:
test1 = forest.predict([arr3])

In [ ]:
print(test1)